# Notebook to Test Database Functions

### Import Functions

In [1]:
from __future__ import print_function
import datetime
import pickle
import os.path
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
from methods import Calendar, Event
from flask_sqlalchemy import SQLAlchemy
from database_run import db, Disciplina, Curso, Departamento, Professor, Sala, Turma, Vinculo

### Initialize Google Calendar API and Database

In [3]:
SCOPES = ['https://www.googleapis.com/auth/calendar']
creds = None
if os.path.exists('token.pickle'):
    with open('token.pickle', 'rb') as token:
        creds = pickle.load(token)
# If there are no (valid) credentials available, let the user log in.
if not creds or not creds.valid:
    if creds and creds.expired and creds.refresh_token:
        creds.refresh(Request())
    else:
        flow = InstalledAppFlow.from_client_secrets_file(
            'json_templates/credentials.json', SCOPES)
        creds = flow.run_local_server()
    # Save the credentials for the next run
    with open('token.pickle', 'wb') as token:
        pickle.dump(creds, token)

calendar_object = Calendar(creds)
event_object =  Event(creds)

# Initialize db
#db.drop_all()
db.create_all()

### Database Functions

In [40]:
class DBM(): 
    # Core methods
    def push(self, data):
        db.session.add(data)
        db.session.commit()
        
    # CRUD Operators
    
    # Database CREATE
    def create_disciplina(self, nome, numero, sigla):
        data = Disciplina(nome, numero, sigla)
        self.push(data)
        return data.id
        
    def create_professor(self, siape, nome, email, telefone, ramal, id_departamento, id_vinculo):
        data = Professor(siape, nome, email, telefone, ramal, id_departamento, id_vinculo)
        self.push(data)
        return data.id
    
    def create_vinculo(self, nome):
        data = Vinculo(nome)
        self.push(data)
        return data.id
        
    def create_departamento(self, nome, sigla, codigo):
        data = Departamento(nome, sigla, codigo)
        self.push(data)
        return data.id
        
    def create_sala(self, nome, capacidade):
        data = Sala(nome, capacidade)
        self.push(data)
        return data.id
        
    def create_turma(self, id_disciplina, id_departamento, numero, codigo=None):
        if codigo == None:
            disciplina = self.find_by_id(Disciplina, id_disciplina)
            departamento = self.find_by_id(Departamento, id_departamento)
            codigo = departamento.sigla + str(disciplina.numero) 
            
        data = Turma(id_disciplina, id_departamento, numero, codigo)
        self.push(data)
        return data.id
        
    # Database UPDATE
    
    # Database DELETE
    
    # General Methods
    def list(self, obj):
        return obj.query.all()

    def find_by_id(self, obj, id):
        return obj.query.filter_by(id=id).first()


### Creating Object

In [41]:
dbm = DBM()

#dbm.create_disciplina('Nome',382,'Sigla')
#dbm.create_departamento('Departamento de Dermato', 'DERM', 40012)
#id_vinculo = dbm.create_vinculo('Aposentado')
#dbm.create_professor(3020423, 'Joaquim', 'email', 'telefone', 'ramal', id_departamento=1, id_vinculo=1)
#dbm.create_sala('F-100', 20)
dbm.create_turma(id_disciplina=1, id_departamento=1, numero=23012)


1

### Listing Object

In [42]:
print('Disciplina', dbm.list(Disciplina))
print('Departamento', dbm.list(Departamento))
print('Professor', dbm.list(Professor))
print('Vinculo', dbm.list(Vinculo))
print('Sala', dbm.list(Sala))
print('Turma', dbm.list(Turma))

Disciplina [<Disciplina 1>]
Departamento [<Departamento 1>]
Professor [<Professor 1>]
Vinculo [<Vinculo 1>]
Sala [<Sala 1>]
Turma [<Turma 1>]
